In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")


# Data Ingestion

In [4]:
from langchain_community.document_loaders import WebBaseLoader

In [7]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Darshan_(Kannada_actor)")
loader

In [8]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Darshan_(Kannada_actor)', 'title': 'Darshan (Kannada actor) - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nDarshan (Kannada actor) - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life\n\n\n\n\n\n\n\n\n2\nActing career\

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_split = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap=200)

documents = text_split.split_documents(docs)


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()



In [ ]:
from langchain_community.vectorstores import FAISS

vs = FAISS.from_documents(documents, embeddings) 

In [ ]:
query = ""


result  = vs.similarity_search(query)

result[0].page_content

#  Retrival Chain

In [ ]:
from langchain_openai import ChatOpenAI
llm  = ChatOpenAI()
print(llm)

In [ ]:
from langchain.chains.chain_documents   import create_stuff_documents_chain

from langchain_core.prompts import ChatPromptTemplate

prompts = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
 """
)

document_chain =  create_stuff_documents_chain(llm, prompts)
document_chain

In [ ]:

from langchain_core.documents import Document  
response = document_chain.invoke({"input":"",
                                  "context":[Document(page_content="")]})

response

In [ ]:
ret = vs.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(ret ,document_chain)